In [1]:
import winglets as wl
import pandas as pd
import proplot as plot

import copy

import numpy as np
import winglets as wl
from Geometry import Point
from winglets.conventions import (
    WingletParameters,
    WingSectionParameters,
    OperationPoint,
)

from tqdm import tqdm

from winglets.optimizer import NAME_CD, NAME_CM
from winglets.utils import get_base_winglet_parametrization, get_bounds

In [2]:
def get_operation_point():

    ALTITUDE = OperationPoint.ALTITUDE.value
    MACH = OperationPoint.MACH.value
    CL = OperationPoint.CL.value

    _dict = {ALTITUDE: 11000, MACH: 0.75, CL: 0.45}

    return _dict

In [3]:
def get_sections():
    """Wing planform sections."""

    WING_AIRFOIL = "naca4412"
    
    CHORD = WingSectionParameters.CHORD.value
    LE_LOCATION = WingSectionParameters.LE_LOCATION.value
    TWIST = WingSectionParameters.TWIST.value
    AIRFOIL = WingSectionParameters.AIRFOIL.value

    _sections = [
        {
            CHORD: 5.6,
            LE_LOCATION: Point([0.0, 0.0, 0.0]),
            TWIST: 0.0,
            AIRFOIL: WING_AIRFOIL,
        },
        {
            CHORD: 3.6,
            LE_LOCATION: Point([2.34, 4.6, 0.2]),
            TWIST: -2.0,
            AIRFOIL: WING_AIRFOIL,
        },
        {
            CHORD: 1.26,
            LE_LOCATION: Point([5.5, 14.04, 0.61]),
            TWIST: -5.0,
            AIRFOIL: WING_AIRFOIL,
        },
    ]

    return _sections

In [4]:
def get_flying_wing(sections):
    """Flying wing with winglets."""
    _wing = wl.FlyingWing(sections=sections, winglet_parameters=None)

    _wing.create_wing_planform()

    return _wing


def get_flying_wing_winglets(sections):
    """Flying wing with winglets."""

    winglet_parameters = get_base_winglet_parametrization()

    _wing = wl.FlyingWing(sections=sections, winglet_parameters=winglet_parameters)

    _wing.create_wing_planform()
    _wing.create_winglet()

    return _wing


In [5]:
_sections = get_sections()
flying_wing = get_flying_wing(sections=_sections)
flying_wing_winglets = get_flying_wing_winglets(sections=_sections)

In [6]:
operation_point = get_operation_point()

In [7]:
initial_winglet = get_base_winglet_parametrization(twist_zero=False)

In [8]:
optimizer = wl.WingletOptimizer(
        base=flying_wing,
        target=flying_wing_winglets,
        operation_point=operation_point,
        initial_winglet=initial_winglet,
        interpolation_factor=0.5,
    )

print(optimizer.MAX_ITER)

# Put up
optimizer.put_up()
_lower, _upper = get_bounds()
optimizer.set_bounds(lower=_lower, upper=_upper)

# Optimize
result = optimizer.optimize()

# Collect
targets, parameters = optimizer.evaluate_optimum()

# Optimization data
J = result.fun
success = optimizer.success
iterations = result.nit
fun_eval = result.nfev
message = result.message

50


In [11]:
result.x 

array([2.        , 1.53846154, 0.9375    , 0.        , 0.33333333,
       2.20818463, 2.77646728])

In [13]:
parameters

{0: 0.1,
 2: 0.3,
 1: 1.0,
 3: 0.0,
 4: 15.0,
 5: 2.208184634308437,
 6: 2.776467278956675,
 'wingletAirfoil': 'naca0012'}

In [20]:
results = []

ks = np.linspace(0, 1.0, 5)

for k in tqdm(ks):
    
    optimizer = wl.WingletOptimizer(
        base=flying_wing,
        target=flying_wing_winglets,
        operation_point=operation_point,
        initial_winglet=initial_winglet,
        interpolation_factor=k,
    )

    optimizer.MAX_ITER = 15

    optimizer.put_up()
    _lower, _upper = get_bounds()
    optimizer.set_bounds(lower=_lower, upper=_upper)
    result = optimizer.optimize()
    targets, parameters = optimizer.evaluate_optimum()

    # Optimization data
    J = result.fun
    success = optimizer.success
    iterations = result.nit
    fun_eval = result.nfev
    message = result.message
    
    cd = targets[NAME_CD]
    cm = targets[NAME_CM]

    # Get winglet parametrization
    winglet = {
        name: parameters[enum_value.value]
        for name, enum_value in WingletParameters.__members__.items()
    }

    span = winglet[WingletParameters.SPAN.name]
    chord_root = winglet[WingletParameters.CHORD_ROOT.name]
    taper_ratio = winglet[WingletParameters.TAPER_RATIO.name]
    sweep = winglet[WingletParameters.ANGLE_SWEEP.name]
    cant = winglet[WingletParameters.ANGLE_CANT.name]
    angle_twist_root = winglet[WingletParameters.ANGLE_TWIST_ROOT.name]
    angle_twist_tip = winglet[WingletParameters.ANGLE_TWIST_TIP.name]

    results.append(
        (
            k,
            success,
            iterations,
            fun_eval,
            message,
            J,
            cd,
            cm,
            span,
            chord_root,
            taper_ratio,
            sweep,
            cant,
            angle_twist_root,
            angle_twist_tip,
        )
    )

100%|██████████| 5/5 [1:56:12<00:00, 1394.46s/it]  


In [21]:
results_df = pd.DataFrame(
    results,
    columns=[
        "k",
        "success",
        "iterations",
        "function_evaluations",
        "message",
        "J",
        "cd",
        "cm",
        WingletParameters.SPAN.name,
        WingletParameters.CHORD_ROOT.name,
        WingletParameters.TAPER_RATIO.name,
        WingletParameters.ANGLE_SWEEP.name,
        WingletParameters.ANGLE_CANT.name,
        WingletParameters.ANGLE_TWIST_ROOT.name,
        WingletParameters.ANGLE_TWIST_TIP.name,
    ],
)

In [22]:
results_df

,k,success,iterations,function_evaluations,message,J,cd,cm,SPAN,CHORD_ROOT,TAPER_RATIO,ANGLE_SWEEP,ANGLE_CANT,ANGLE_TWIST_ROOT,ANGLE_TWIST_TIP
0,0.00,False,10,608,b'ABNORMAL_TERMINATION_IN_LNSRCH',-53.457461,-9.181790,2755.559005,0.02,0.922354,0.300004,50.000000,89.999596,4.017695,2.492942
1,0.25,False,15,992,b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT',-14.215107,-0.337795,-54.175338,0.10,0.400000,0.998897,0.000132,89.999183,4.684917,4.684950
2,0.50,False,15,168,b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT',0.925975,0.004614,-53.205201,0.10,1.000000,0.301575,0.000000,15.000000,2.166370,2.355042
3,0.75,False,15,168,b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT',0.872108,0.004604,-53.338228,0.10,1.000000,0.406487,0.000000,15.000000,2.672954,1.799494
4,1.00,False,15,160,b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT',0.817633,0.004602,-53.429563,0.10,1.000000,0.536995,0.000000,15.000000,2.999832,1.243204


In [ ]:
fig, axes = plot.subplots(array=[[1, 2], [3, 2]], sharex=False, sharey=False)

options = dict(marker="o")

axes[0].plot(results_df["$\\phi$"], **options)
axes[0].set_ylabel("$\\phi$ [deg]")

axes[1].scatter(
    results_df["cd"].values / optimizer.base_results[NAME_CD],
    results_df["cm"].values / optimizer.base_results[NAME_CM],
    c=results_df["J"],
)

axes[1].set_ylabel("$C_M$ / $C_{M0}$")
axes[1].set_xlabel("$C_d$ / $C_{d0}$")

axes[2].plot(results_df["J"], **options, c="purple")
axes[2].set_ylabel("J")

axes[0].format(abc=True)
axes[1].format(abc=True)
axes[2].format(abc=True)

fig.save("1d_optimization_results.png", dpi=300, bbox_inches="tight", transparent=True)